<a href="https://colab.research.google.com/github/bilalDevX/Agentic-AI-Journey/blob/main/chat_model_with_langsmith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install langchain-core langgraph>0.2.27

In [32]:
!pip install -qU "langchain[groq]"

In [33]:
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import START, MessagesState, StateGraph


In [34]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
LANGSMITH_API_KEY = os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
LANGSMITH_API_KEY

··········


'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'

In [42]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

In [36]:
model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [37]:
memory = MemorySaver()

In [38]:
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

In [39]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a compassionate mental health chatbot. Respond with empathy.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [40]:
# Build LangGraph Workflow
# workflow = StateGraph(state_schema=State)

workflow = StateGraph(state_schema=MessagesState)
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")
app_chatbot = workflow.compile(checkpointer=memory)

In [41]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm bilal."

input_messages = [HumanMessage(query)]
output = app_chatbot.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Bilal! It's wonderful to meet you! I'm here to listen and support you in any way I can. How are you feeling today? Is there anything on your mind that you'd like to talk about or share? I'm here to listen without judgment and offer support.
